# Lab 3-2 Part 1: Model Drift Monitoring Pipeline

## 📋 실습 개요

| 항목 | 내용 |
|------|------|
| **소요시간** | 45분 |
| **난이도** | ⭐⭐⭐ |
| **목표** | Prometheus/Grafana를 활용한 Model Drift 모니터링 |
| **사전 조건** | Monitoring Stack 배포됨 |

## 🎯 학습 목표

1. Prometheus 메트릭 조회 및 이해
2. Model Drift 감지를 위한 메트릭 분석
3. Alert Rule 설정 및 테스트
4. Grafana 대시보드 활용

---

## Step 0: 환경 설정

In [ ]:
# 필수 패키지 설치
!pip install requests pandas matplotlib seaborn -q
print("✅ 패키지 설치 완료")

In [ ]:
import os
import json
import time
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Plot 스타일
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print("✅ Import 완료")

In [ ]:
# ============================================================
# ⚠️ 본인의 사용자 번호로 변경하세요!
# ============================================================
USER_NUM = "YOUR_USER_NUM"  # 예: "01", "02", ... "15", "20"

# 환경 설정
USER_ID = f"user{USER_NUM}"
NAMESPACE = f"kubeflow-{USER_ID}"

# Prometheus/Grafana URL
PROMETHEUS_URL = "http://prometheus.monitoring.svc.cluster.local:9090"
GRAFANA_URL = "http://grafana.monitoring.svc.cluster.local:3000"

# 메트릭 임계값
MAE_THRESHOLD = 0.45       # MAE가 이 값 초과 시 Drift
R2_THRESHOLD = 0.75        # R²가 이 값 미만 시 Drift
ERROR_RATE_THRESHOLD = 0.05  # 에러율 5% 초과 시 Alert

print("="*60)
print("  Lab 3-2 Part 1: Model Drift Monitoring")
print("="*60)
print(f"\n👤 사용자: {USER_ID}")
print(f"📁 네임스페이스: {NAMESPACE}")
print(f"\n📊 Drift 임계값:")
print(f"   MAE > {MAE_THRESHOLD}")
print(f"   R² < {R2_THRESHOLD}")
print(f"   Error Rate > {ERROR_RATE_THRESHOLD*100}%")

---

## Step 1: Monitoring Stack 연결 확인

**⚠️ 이 셀 실행 전에 터미널에서 포트포워딩을 실행하세요:**

```bash
kubectl port-forward -n monitoring svc/prometheus 9090:9090 &
kubectl port-forward -n monitoring svc/grafana 3000:3000 &
```

In [ ]:
def check_prometheus_connection():
    """Prometheus 연결 확인"""
    try:
        response = requests.get(f"{PROMETHEUS_URL}/-/healthy", timeout=5)
        return response.status_code == 200
    except:
        return False

def check_grafana_connection():
    """Grafana 연결 확인"""
    try:
        response = requests.get(f"{GRAFANA_URL}/api/health", timeout=5)
        return response.status_code == 200
    except:
        return False

print("="*60)
print("  Monitoring Stack 연결 확인")
print("="*60)

# Prometheus 확인
if check_prometheus_connection():
    print(f"\n✅ Prometheus 연결 성공: {PROMETHEUS_URL}")
else:
    print(f"\n❌ Prometheus 연결 실패")
    print("   터미널에서 실행: kubectl port-forward -n monitoring svc/prometheus 9090:9090")

# Grafana 확인
if check_grafana_connection():
    print(f"✅ Grafana 연결 성공: {GRAFANA_URL}")
else:
    print(f"❌ Grafana 연결 실패")
    print("   터미널에서 실행: kubectl port-forward -n monitoring svc/grafana 3000:3000")

---

## Step 2: Prometheus 메트릭 조회

### 2.1 메트릭 조회 함수 정의

In [ ]:
def query_prometheus(query):
    """Prometheus instant query 실행"""
    try:
        response = requests.get(
            f"{PROMETHEUS_URL}/api/v1/query",
            params={"query": query},
            timeout=10
        )
        if response.status_code == 200:
            data = response.json()
            if data["status"] == "success":
                return data["data"]["result"]
    except Exception as e:
        print(f"Query error: {e}")
    return []

def query_prometheus_range(query, start_time, end_time, step="15s"):
    """Prometheus range query 실행"""
    try:
        response = requests.get(
            f"{PROMETHEUS_URL}/api/v1/query_range",
            params={
                "query": query,
                "start": start_time.isoformat() + "Z",
                "end": end_time.isoformat() + "Z",
                "step": step
            },
            timeout=30
        )
        if response.status_code == 200:
            data = response.json()
            if data["status"] == "success":
                return data["data"]["result"]
    except Exception as e:
        print(f"Range query error: {e}")
    return []

print("✅ Prometheus 쿼리 함수 정의 완료")

### 2.2 현재 모델 메트릭 조회

In [ ]:
print("="*60)
print("  현재 모델 메트릭 조회")
print("="*60)

# MAE Score
print("\n📊 model_mae_score (Mean Absolute Error):")
mae_results = query_prometheus('model_mae_score')
if mae_results:
    for r in mae_results[:10]:  # 상위 10개
        user_id = r["metric"].get("user_id", "unknown")
        value = float(r["value"][1])
        status = "⚠️" if value > MAE_THRESHOLD else "✅"
        print(f"   {status} {user_id}: {value:.4f}")
else:
    print("   (데이터 없음)")

# R² Score
print("\n📊 model_r2_score (R² Score):")
r2_results = query_prometheus('model_r2_score')
if r2_results:
    for r in r2_results[:10]:
        user_id = r["metric"].get("user_id", "unknown")
        value = float(r["value"][1])
        status = "⚠️" if value < R2_THRESHOLD else "✅"
        print(f"   {status} {user_id}: {value:.4f}")
else:
    print("   (데이터 없음)")

### 2.3 본인 메트릭만 조회

In [ ]:
print("="*60)
print(f"  {USER_ID} 메트릭 상세 조회")
print("="*60)

# 본인 MAE
my_mae = query_prometheus(f'model_mae_score{{user_id="{USER_ID}"}}')
if my_mae:
    mae_value = float(my_mae[0]["value"][1])
    print(f"\n📉 MAE Score: {mae_value:.4f}")
    if mae_value > MAE_THRESHOLD:
        print(f"   ⚠️ 임계값({MAE_THRESHOLD}) 초과! Drift 의심")
    else:
        print(f"   ✅ 정상 범위")

# 본인 R²
my_r2 = query_prometheus(f'model_r2_score{{user_id="{USER_ID}"}}')
if my_r2:
    r2_value = float(my_r2[0]["value"][1])
    print(f"\n📈 R² Score: {r2_value:.4f}")
    if r2_value < R2_THRESHOLD:
        print(f"   ⚠️ 임계값({R2_THRESHOLD}) 미만! Drift 의심")
    else:
        print(f"   ✅ 정상 범위")

# 예측 처리량
my_rps = query_prometheus(f'rate(model_prediction_total{{user_id="{USER_ID}", status="success"}}[5m])')
if my_rps:
    rps_value = float(my_rps[0]["value"][1])
    print(f"\n⚡ Prediction Rate: {rps_value:.2f} req/sec")

---

## Step 3: 메트릭 시계열 분석

지난 30분간의 메트릭 변화를 시각화합니다.

In [ ]:
# 시간 범위 설정 (최근 30분)
end_time = datetime.utcnow()
start_time = end_time - timedelta(minutes=30)

print(f"📅 조회 기간: {start_time.strftime('%H:%M:%S')} ~ {end_time.strftime('%H:%M:%S')} UTC")

# MAE 시계열 데이터
mae_range = query_prometheus_range(
    f'model_mae_score{{user_id="{USER_ID}"}}',
    start_time, end_time, step="30s"
)

# R² 시계열 데이터
r2_range = query_prometheus_range(
    f'model_r2_score{{user_id="{USER_ID}"}}',
    start_time, end_time, step="30s"
)

if mae_range and r2_range:
    print(f"✅ {len(mae_range[0]['values'])} 데이터 포인트 조회됨")
else:
    print("⚠️ 시계열 데이터가 없습니다. Metrics Exporter가 실행 중인지 확인하세요.")

In [ ]:
# 시계열 시각화
if mae_range and r2_range:
    fig, axes = plt.subplots(2, 1, figsize=(12, 8))
    
    # MAE 차트
    mae_times = [datetime.fromtimestamp(float(v[0])) for v in mae_range[0]['values']]
    mae_values = [float(v[1]) for v in mae_range[0]['values']]
    
    axes[0].plot(mae_times, mae_values, 'b-', linewidth=2, label='MAE')
    axes[0].axhline(y=MAE_THRESHOLD, color='r', linestyle='--', label=f'Threshold ({MAE_THRESHOLD})')
    axes[0].fill_between(mae_times, MAE_THRESHOLD, max(mae_values + [MAE_THRESHOLD]), 
                         alpha=0.2, color='red', label='Drift Zone')
    axes[0].set_title(f'{USER_ID} - Model MAE Score (The lower the better)', fontsize=14)
    axes[0].set_ylabel('MAE')
    axes[0].legend(loc='upper right')
    axes[0].grid(True, alpha=0.3)
    
    # R² 차트
    r2_times = [datetime.fromtimestamp(float(v[0])) for v in r2_range[0]['values']]
    r2_values = [float(v[1]) for v in r2_range[0]['values']]
    
    axes[1].plot(r2_times, r2_values, 'g-', linewidth=2, label='R²')
    axes[1].axhline(y=R2_THRESHOLD, color='r', linestyle='--', label=f'Threshold ({R2_THRESHOLD})')
    axes[1].fill_between(r2_times, 0, R2_THRESHOLD, 
                         alpha=0.2, color='red', label='Drift Zone')
    axes[1].set_title(f'{USER_ID} - Model R² Score (The higher the better)', fontsize=14)
    axes[1].set_ylabel('R²')
    axes[1].set_xlabel('Time')
    axes[1].legend(loc='lower right')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Drift 판정
    current_mae = mae_values[-1] if mae_values else 0
    current_r2 = r2_values[-1] if r2_values else 1
    
    print("\n" + "="*60)
    print("  Drift 판정 결과")
    print("="*60)
    print(f"\n현재 MAE: {current_mae:.4f} (임계값: {MAE_THRESHOLD})")
    print(f"현재 R²:  {current_r2:.4f} (임계값: {R2_THRESHOLD})")
    
    drift_detected = current_mae > MAE_THRESHOLD or current_r2 < R2_THRESHOLD
    if drift_detected:
        print("\n🚨 DRIFT 감지됨! 모델 재학습이 필요합니다.")
    else:
        print("\n✅ 모델 성능 정상")
else:
    print("⚠️ 시각화할 데이터가 없습니다.")

---

## Step 4: 전체 사용자 비교 분석

In [ ]:
# 전체 사용자 메트릭 수집
all_mae = query_prometheus('model_mae_score')
all_r2 = query_prometheus('model_r2_score')

if all_mae and all_r2:
    # DataFrame 생성
    mae_df = pd.DataFrame([
        {"user_id": r["metric"].get("user_id", "unknown"), "mae": float(r["value"][1])}
        for r in all_mae
    ])
    
    r2_df = pd.DataFrame([
        {"user_id": r["metric"].get("user_id", "unknown"), "r2": float(r["value"][1])}
        for r in all_r2
    ])
    
    # 병합
    metrics_df = mae_df.merge(r2_df, on="user_id")
    metrics_df = metrics_df.sort_values("user_id")
    
    # Drift 판정
    metrics_df["drift"] = (metrics_df["mae"] > MAE_THRESHOLD) | (metrics_df["r2"] < R2_THRESHOLD)
    
    print("="*60)
    print("  전체 사용자 메트릭 비교")
    print("="*60)
    print(f"\n총 사용자: {len(metrics_df)}명")
    print(f"Drift 감지: {metrics_df['drift'].sum()}명")
    print("\n" + metrics_df.to_string(index=False))

In [ ]:
# 시각화
if 'metrics_df' in dir() and len(metrics_df) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # MAE 비교
    colors = ['red' if d else 'green' for d in metrics_df['drift']]
    axes[0].barh(metrics_df['user_id'], metrics_df['mae'], color=colors, alpha=0.7)
    axes[0].axvline(x=MAE_THRESHOLD, color='red', linestyle='--', linewidth=2, label=f'Threshold ({MAE_THRESHOLD})')
    axes[0].set_xlabel('MAE')
    axes[0].set_title('Model MAE by User (The lower the better)')
    axes[0].legend()
    
    # R² 비교
    axes[1].barh(metrics_df['user_id'], metrics_df['r2'], color=colors, alpha=0.7)
    axes[1].axvline(x=R2_THRESHOLD, color='red', linestyle='--', linewidth=2, label=f'Threshold ({R2_THRESHOLD})')
    axes[1].set_xlabel('R²')
    axes[1].set_title('Model R² by User (The higher the better)')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    # 본인 강조
    my_row = metrics_df[metrics_df['user_id'] == USER_ID]
    if len(my_row) > 0:
        print(f"\n🎯 {USER_ID}의 위치:")
        print(f"   MAE: {my_row['mae'].values[0]:.4f} (전체 순위: {(metrics_df['mae'] < my_row['mae'].values[0]).sum() + 1}/{len(metrics_df)})")
        print(f"   R²:  {my_row['r2'].values[0]:.4f} (전체 순위: {(metrics_df['r2'] > my_row['r2'].values[0]).sum() + 1}/{len(metrics_df)})")

---

## Step 5: Alert 상태 확인

In [ ]:
def get_prometheus_alerts():
    """Prometheus Alert 상태 조회"""
    try:
        response = requests.get(f"{PROMETHEUS_URL}/api/v1/alerts", timeout=10)
        if response.status_code == 200:
            return response.json().get("data", {}).get("alerts", [])
    except Exception as e:
        print(f"Alert query error: {e}")
    return []

print("="*60)
print("  Prometheus Alert 상태")
print("="*60)

alerts = get_prometheus_alerts()
if alerts:
    print(f"\n🚨 활성 Alert: {len(alerts)}개\n")
    for alert in alerts:
        name = alert.get("labels", {}).get("alertname", "Unknown")
        state = alert.get("state", "unknown")
        user_id = alert.get("labels", {}).get("user_id", "N/A")
        severity = alert.get("labels", {}).get("severity", "unknown")
        
        emoji = "🔴" if state == "firing" else "🟡" if state == "pending" else "🟢"
        print(f"  {emoji} [{severity}] {name}")
        print(f"      User: {user_id}, State: {state}")
        print()
else:
    print("\n✅ 활성 Alert 없음")

---

## Step 6: Drift 감지 요약 리포트

In [ ]:
def generate_drift_report(user_id):
    """사용자별 Drift 분석 리포트 생성"""
    print("="*60)
    print(f"  Drift Analysis Report - {user_id}")
    print("="*60)
    print(f"\n📅 생성 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # 현재 메트릭
    mae_result = query_prometheus(f'model_mae_score{{user_id="{user_id}"}}')
    r2_result = query_prometheus(f'model_r2_score{{user_id="{user_id}"}}')
    
    if not mae_result or not r2_result:
        print("\n⚠️ 메트릭 데이터를 가져올 수 없습니다.")
        return None
    
    current_mae = float(mae_result[0]["value"][1])
    current_r2 = float(r2_result[0]["value"][1])
    
    print("\n📊 현재 메트릭:")
    print(f"   MAE: {current_mae:.4f}")
    print(f"   R²:  {current_r2:.4f}")
    
    print("\n📏 임계값:")
    print(f"   MAE Threshold: {MAE_THRESHOLD}")
    print(f"   R² Threshold:  {R2_THRESHOLD}")
    
    # Drift 판정
    mae_drift = current_mae > MAE_THRESHOLD
    r2_drift = current_r2 < R2_THRESHOLD
    overall_drift = mae_drift or r2_drift
    
    print("\n🔍 Drift 분석:")
    print(f"   MAE Drift: {'⚠️ YES' if mae_drift else '✅ NO'} ({current_mae:.4f} {'>' if mae_drift else '<='} {MAE_THRESHOLD})")
    print(f"   R² Drift:  {'⚠️ YES' if r2_drift else '✅ NO'} ({current_r2:.4f} {'<' if r2_drift else '>='} {R2_THRESHOLD})")
    
    print("\n" + "="*60)
    if overall_drift:
        print("  🚨 결론: DRIFT 감지됨 - 재학습 권장")
        print("="*60)
        print("\n권장 조치:")
        print("  1. 데이터 분포 변화 확인")
        print("  2. 새 데이터로 모델 재학습")
        print("  3. A/B 테스트 후 신규 모델 배포")
    else:
        print("  ✅ 결론: 모델 성능 정상")
        print("="*60)
    
    return {
        "user_id": user_id,
        "timestamp": datetime.now().isoformat(),
        "mae": current_mae,
        "r2": current_r2,
        "drift_detected": overall_drift
    }

# 리포트 생성
report = generate_drift_report(USER_ID)

---

## ✅ Part 1 완료 체크리스트

- [ ] Prometheus 연결 성공
- [ ] 본인 메트릭 조회 완료 (MAE, R²)
- [ ] 시계열 차트 확인
- [ ] 전체 사용자 비교 분석
- [ ] Alert 상태 확인
- [ ] Drift 리포트 생성

---

## 📚 다음 단계

**Part 2: CI/CD Pipeline**에서는:
- GitHub Actions 워크플로우 분석
- 자동 재학습 트리거 구현
- End-to-End 통합 테스트

→ `lab3-2_part2_cicd.ipynb` 실행